# Training

In [146]:
import pandas as pd
import numpy as np
X = pd.read_csv('data.csv')
y = pd.read_csv('marks.csv').values
y = np.reshape(y, (y.shape[0],))
X['Anaph:index_text'] = X['Anaph:noun_index']

In [147]:
keys = ['sent_num', 'index_text', 'index_sent', 'start_symb', 'end_symb']
print(X['Anaph:Lemma'].unique())
lemmas = ['они', 'его', 'их', 'он', 'ее', 'она', 'тот', 'оно']
X['Answer'] = y
X=X[X['Anaph:Lemma'].isin(lemmas)]
del X['Anaph:Lemma'], X['Ant:Lemma']
for key in keys:
    X['diff_'+key] = X['Anaph:'+key]-X['Ant:'+key]
X['index'] = X['Anaph:'+keys[0]]
X['index'] = X['Anaph:index_text']
print(len(X['Anaph:index_text'].unique()))
for key in keys:
    del X['Anaph:' + key], X['Ant:'+key]
y = X['Answer'].values
X.head(10)

['который' 'они' 'себя' 'его' 'их' 'он' 'ее' 'она' 'свой' 'тот' 'оно' 'я'
 'кто' 'мой']
801


,Ant:ParentMorph:Animacy:Anim,Ant:ParentMorph:Animacy:Inan,Ant:ParentMorph:Aspect:Imp,Ant:ParentMorph:Aspect:Perf,Ant:ParentMorph:Case:Acc,Ant:ParentMorph:Case:Dat,Ant:ParentMorph:Case:Gen,Ant:ParentMorph:Case:Ins,Ant:ParentMorph:Case:Loc,Ant:ParentMorph:Case:Nom,...,Anaph:role:транзитив,Anaph:role:трансгрессив,Anaph:role:фабрикатив,Answer,diff_sent_num,diff_index_text,diff_index_sent,diff_start_symb,diff_end_symb,index
5,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,3,98,40,683,679,111
6,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,3,97,39,675,669,111
7,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,3,91,33,646,639,111
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,89,39,624,621,111
9,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,2,82,32,583,581,111
10,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,2,78,28,572,569,111
11,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,2,76,26,564,559,111
12,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,2,72,22,538,538,111
13,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,2,61,11,488,479,111
14,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,2,60,10,475,473,111


In [148]:
index, dist = X['index'].values, X['diff_' + 'index_sent'].values
res = {i:[] for i in index}
for ind, i in enumerate(index):
    res[i].append((dist[ind],ind))
def transform(res):
    list_ = []
    u = []
    for i in res:
        cur = res[i]
        cur.sort(key = lambda x: x[0])
        u.append(len(cur))
        list_ += [(ind, i[1]) for ind,i in enumerate(cur)]
    res = list_.copy()
    for i in list_:
        res[i[1]] = i[0]
    return res
X['Distance'] = transform(res)
del X['index']
keys = X.columns

In [149]:
import numpy as np
X = X.values
y = np.reshape(y, (y.shape[0],))

In [150]:
def my_train_test_split(X,y,index, size = 0.1):
    indexes = np.unique(index)
    size = int(len(indexes)*size)
    choice = np.random.choice(indexes, size)
    X_test, y_test, X_train, y_train = [], [], [], []
    index_test, index_train = [], []
    for ind,i in enumerate(index):
        if i in choice:
            X_test.append(X[ind])
            y_test.append(y[ind])
            index_test.append(i)
        else:
            X_train.append(X[ind])
            y_train.append(y[ind])
            index_train.append(i)
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    index_train = np.array(index_train)
    index_test = np.array(index_test)
    return X_train, X_test, y_train, y_test, index_train, index_test
s = 0
for i in y:
    if i == 1:
        s += 1
print(s, len(y))
a = []
for i in index:
    if not i in a:
        a.append(i)
print(len(a))

347 17923
801


# Gradient Boosting on Trees

In [192]:
X_train, X_test, y_train, y_test, index_train, index_test = my_train_test_split(X,y, index)

In [196]:
import lightgbm as lgb
param = {'num_leaves': 4,
         'objective': 'binary',
         'metric' : ''}
num_round = 1000
train_data = lgb.Dataset(X_train, label=y_train)
bst = lgb.train(param,train_data, num_round)

In [197]:
def metric(y_pred, y, index):
    res =dict()
    S = 0
    true = dict()
    for ind,i in enumerate(y):
        if i==1:
            true[index[ind]] = ind
            res[index[ind]] = []
            S += 1
    for ind, i in enumerate(y_pred):
        if index[ind] in res:
            res[index[ind]].append((ind,i))
    s = 0
    for i in res:
        s += int(max(res[i],key = lambda x: x[1])[0] == true[i])
    return s/S

In [198]:
from sklearn.metrics import f1_score
print(X_test.shape)
y_pred = bst.predict(X_test)
index_test = [int(i) for i in index_test]
metric(y_pred, y_test, index_test)

(1809, 337)


0.75

# Logistic Regression

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
lsvc = LinearSVC(C=1000, penalty="l1", dual=False, max_iter = 10000).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
X_new.shape

In [ ]:
keys[model.get_support()]

In [88]:
from sklearn.preprocessing import scale, PolynomialFeatures
X = scale(X)
index =np.reshape(index, (index.shape[0],1))
X = np.hstack((X, index))
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
index_train, index_test = X_train[:,-1], X_test[:,-1]
X_train, X_test = X_train[:,:-1], X_test[:,:-1]
X_train.shape

/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


(28192, 336)

In [12]:
print(len([i for i in y_test if i == 1]))
print(len([i for i in y_train if i == 1]))

74
474
